# Model 8: Additional Capacity

This notebook contains the construction and training of the model iterations and various experiments. The notebook is split up into four sections: training mode selection (where the model will run), set-up, model constrution, and training. 

Evaluation will take place in the *model_optimization_and_evaluation.ipynb* notebook found in the *notebooks* folder.

In [1]:
# importing necessary packages and libraries
import numpy as np
import pandas as pd
from keras import layers
from keras import models
from keras import optimizers
from keras import applications 
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint
import pickle

Using TensorFlow backend.


## Step 1: Training Mode Selection

In the cell below, specify the training mode for the model. This will determine the location from which the source data is drawn, and to which the trained models (and training histories) are saved. 

- **training_mode = 'floydhub'** (runs on Floydhub)
- **training_mode = 'local'** (runs on local disk and processor)

In [2]:
# select training mode
training_mode = 'floydhub'

## Step 2: Set-up

In [3]:
# directory base paths
data_path_local = '../../data/0002_array_data/train_data/'
model_path_local = '../../notebooks/model_construction/saved_models/'
data_path_floydhub = '/floyd/input/capstone_mushrooms/'
model_path_floydhub = '/floyd/home/'

# setting directory paths based on training mode selection
if training_mode == 'floydhub':
    data_path = data_path_floydhub
    model_path = model_path_floydhub
elif training_mode == 'local':
    data_path = data_path_local
    model_path = model_path_local
else:
    raise Exception('Please choose valid training mode: "floydhub" or "local".')

In [4]:
# loading the training and validation data subsets
X_train = np.load(f'{data_path}X_train_data.npy')
y_train = np.load(f'{data_path}y_train_data.npy')
X_val = np.load(f'{data_path}X_val_data.npy')
y_val = np.load(f'{data_path}y_val_data.npy')

In [5]:
# setting training parameters
batch_size = 8
n_classes = 20
n_epochs = 30
img_shape = X_train.shape[1:]
model_names = []
model_list = []
model_hists = []

## Step 3: Model Construction

### Trial 1 - Additional convolutions

In [6]:
# defining the model architecture
m8_t1 = models.Sequential()

# convolution/max pool stacks
m8_t1.add(layers.Conv2D(32,(3,3), input_shape=img_shape, padding='same'))
m8_t1.add(layers.LeakyReLU(alpha=0.1))
m8_t1.add(layers.MaxPooling2D((2,2)))

m8_t1.add(layers.Conv2D(64,(3,3), padding='same'))
m8_t1.add(layers.LeakyReLU(alpha=0.1))
m8_t1.add(layers.MaxPooling2D((2,2)))

m8_t1.add(layers.Conv2D(128,(3,3), padding='same'))
m8_t1.add(layers.LeakyReLU(alpha=0.1))
m8_t1.add(layers.MaxPooling2D((2,2)))

m8_t1.add(layers.Conv2D(256,(3,3), padding='same'))
m8_t1.add(layers.LeakyReLU(alpha=0.1))
m8_t1.add(layers.MaxPooling2D((2,2)))

m8_t1.add(layers.Conv2D(512,(3,3), padding='same'))
m8_t1.add(layers.LeakyReLU(alpha=0.1))
m8_t1.add(layers.MaxPooling2D((2,2)))

# fully connected layers
m8_t1.add(layers.Flatten())
m8_t1.add(layers.Dense(1024))
m8_t1.add(layers.LeakyReLU(alpha=0.1))
m8_t1.add(layers.Dense(n_classes, activation='softmax'))

# reviewing the model architecture and adding model and name to list
m8_t1.summary()
model_names.append('m8_t1')
model_list.append(m8_t1)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 200, 200, 32)      896       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 200, 200, 32)      0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 100, 100, 32)      0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 100, 100, 64)      18496     
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 100, 100, 64)      0         
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 50, 50, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 50, 50, 128)      

### Trial 2 - Additional convolutions and replacing final max pooling

In [7]:
# defining the model architecture
m8_t2 = models.Sequential()

# convolution/max pool stacks
m8_t2.add(layers.Conv2D(32,(3,3), input_shape=img_shape, padding='same'))
m8_t2.add(layers.LeakyReLU(alpha=0.1))
m8_t2.add(layers.MaxPooling2D((2,2)))

m8_t2.add(layers.Conv2D(64,(3,3), padding='same'))
m8_t2.add(layers.LeakyReLU(alpha=0.1))
m8_t2.add(layers.MaxPooling2D((2,2)))

m8_t2.add(layers.Conv2D(128,(3,3), padding='same'))
m8_t2.add(layers.LeakyReLU(alpha=0.1))
m8_t2.add(layers.MaxPooling2D((2,2)))

m8_t2.add(layers.Conv2D(256,(3,3), padding='same'))
m8_t2.add(layers.LeakyReLU(alpha=0.1))
m8_t2.add(layers.MaxPooling2D((2,2)))

m8_t2.add(layers.Conv2D(512,(3,3), padding='same'))
m8_t2.add(layers.LeakyReLU(alpha=0.1))
m8_t2.add(layers.Conv2D(512,(3,3), strides=2, padding='same'))
m8_t2.add(layers.LeakyReLU(alpha=0.1))

# fully connected layers
m8_t2.add(layers.Flatten())
m8_t2.add(layers.Dense(1024))
m8_t2.add(layers.LeakyReLU(alpha=0.1))
m8_t2.add(layers.Dense(n_classes, activation='softmax'))

# reviewing the model architecture and adding model and name to list
m8_t2.summary()
model_names.append('m8_t2')
model_list.append(m8_t2)

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 200, 200, 32)      896       
_________________________________________________________________
leaky_re_lu_7 (LeakyReLU)    (None, 200, 200, 32)      0         
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 100, 100, 32)      0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 100, 100, 64)      18496     
_________________________________________________________________
leaky_re_lu_8 (LeakyReLU)    (None, 100, 100, 64)      0         
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 50, 50, 64)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 50, 50, 128)      

## Step 4: Training

In [8]:
# setting up standardization and augmentation parameters
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=40,
                                   width_shift_range=0.3,
                                   height_shift_range=0.3,
                                   shear_range=0.3,
                                   zoom_range=0.3,
                                   fill_mode='nearest',
                                   horizontal_flip=True,
                                   vertical_flip=True)
val_datagen = ImageDataGenerator(rescale=1./255)

In [9]:
%%time
# data standardization and augmentation
train_generator = train_datagen.flow(X_train, y_train, batch_size=batch_size)
val_generator = val_datagen.flow(X_val, y_val, batch_size=batch_size)

CPU times: user 329 ms, sys: 1.03 s, total: 1.36 s
Wall time: 1.36 s


In [10]:
# compiling loss functions
m8_t1.compile(loss='categorical_crossentropy', optimizer='adamax', metrics=['acc'])
m8_t2.compile(loss='categorical_crossentropy', optimizer='adamax', metrics=['acc'])

In [11]:
%%time
# setting up model saving checkpoints
m8_t1_cp = ModelCheckpoint(filepath=f'{model_path}m8_t1.h5',
                              monitor='val_acc',
                              verbose=1,
                              save_best_only=True)

# fitting model
m8_t1_history = m8_t1.fit(train_generator,
                                steps_per_epoch=len(X_train)//batch_size,
                                epochs=n_epochs,
                                callbacks=[m8_t1_cp],
                                validation_data=val_generator,
                                validation_steps=len(X_val)//batch_size)

# adding training history to list
model_hists.append(m8_t1_history)

Epoch 1/30
1013/1013 [==============================] - 80s 79ms/step - loss: 2.6873 - acc: 0.1356 - val_loss: 1.7893 - val_acc: 0.2277

Epoch 00001: val_acc improved from -inf to 0.22772, saving model to /floyd/home/m8_t1.h5
Epoch 2/30
1013/1013 [==============================] - 79s 78ms/step - loss: 2.3853 - acc: 0.2293 - val_loss: 3.0106 - val_acc: 0.3017

Epoch 00002: val_acc improved from 0.22772 to 0.30175, saving model to /floyd/home/m8_t1.h5
Epoch 3/30
1013/1013 [==============================] - 79s 78ms/step - loss: 2.2298 - acc: 0.2756 - val_loss: 2.5752 - val_acc: 0.4127

Epoch 00003: val_acc improved from 0.30175 to 0.41272, saving model to /floyd/home/m8_t1.h5
Epoch 4/30
1013/1013 [==============================] - 79s 78ms/step - loss: 2.1163 - acc: 0.3189 - val_loss: 1.9225 - val_acc: 0.4177

Epoch 00004: val_acc improved from 0.41272 to 0.41771, saving model to /floyd/home/m8_t1.h5
Epoch 5/30
1013/1013 [==============================] - 79s 78ms/step - loss: 2.0124 - 

In [12]:
%%time
# setting up model saving checkpoints
m8_t2_cp = ModelCheckpoint(filepath=f'{model_path}m8_t2.h5',
                              monitor='val_acc',
                              verbose=1,
                              save_best_only=True)

# fitting model
m8_t2_history = m8_t2.fit(train_generator,
                                steps_per_epoch=len(X_train)//batch_size,
                                epochs=n_epochs,
                                callbacks=[m8_t2_cp],
                                validation_data=val_generator,
                                validation_steps=len(X_val)//batch_size)

# adding training history to list
model_hists.append(m8_t2_history)

Epoch 1/30
1013/1013 [==============================] - 79s 78ms/step - loss: 2.7914 - acc: 0.1095 - val_loss: 2.5518 - val_acc: 0.1708

Epoch 00001: val_acc improved from -inf to 0.17079, saving model to /floyd/home/m8_t2.h5
Epoch 2/30
1013/1013 [==============================] - 80s 79ms/step - loss: 2.5627 - acc: 0.1591 - val_loss: 2.2397 - val_acc: 0.2095

Epoch 00002: val_acc improved from 0.17079 to 0.20948, saving model to /floyd/home/m8_t2.h5
Epoch 3/30
1013/1013 [==============================] - 79s 78ms/step - loss: 2.4284 - acc: 0.2036 - val_loss: 2.1590 - val_acc: 0.2032

Epoch 00003: val_acc did not improve from 0.20948
Epoch 4/30
1013/1013 [==============================] - 80s 79ms/step - loss: 2.3058 - acc: 0.2522 - val_loss: 2.1109 - val_acc: 0.2618

Epoch 00004: val_acc improved from 0.20948 to 0.26185, saving model to /floyd/home/m8_t2.h5
Epoch 5/30
1013/1013 [==============================] - 80s 79ms/step - loss: 2.1857 - acc: 0.2865 - val_loss: 2.2449 - val_acc: 

In [13]:
# creating dictionary for model names, models, and histories from respective lists
models_dict = {i:[j,k] for i,j,k in zip(model_names,model_list,model_hists)}

In [14]:
# evaluating models on validation set
for key, value in models_dict.items():
    model = models.load_model(f'{model_path}{key}.h5')
    (val_loss, val_accuracy) = model.evaluate(val_generator,verbose=1)
    print(f'{key} Val Accuracy: {round((val_accuracy*100),2)}%')
    print(f'{key} Val Loss: {round(val_loss,4)}')
    print('---')

102/102 [==============================] - 2s 20ms/step
m8_t1 Val Accuracy: 70.25%
m8_t1 Val Loss: 0.1861
---
102/102 [==============================] - 2s 21ms/step
m8_t2 Val Accuracy: 64.07%
m8_t2 Val Loss: 0.0824
---


In [15]:
# saving training histories
for key, value in models_dict.items():
    with open(f'{model_path}{key}_history', 'wb') as file_pi:
        pickle.dump(value[1].history, file_pi)
    print(f'{key}_history saved in {model_path}')

m8_t1_history saved in /floyd/home/
m8_t2_history saved in /floyd/home/
